<a href="https://colab.research.google.com/github/johntango/shakespeare001/blob/main/ShakespeareColab_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [102]:
%%bigquery --project demoproject002-351320 --verbose df
SELECT * FROM `demoproject002-351320.Shakespeare.shakespeare`

Executing query with job ID: 8b2ecc09-2d13-457f-88c3-abfb06af5655
Query executing: 21.99s
Query complete after 22.86s


In [103]:
df.head

<bound method NDFrame.head of                                            string_field_0 string_field_1  \
0                                             THE SONNETS           None   
1                                                    None           None   
2                                  by William Shakespeare           None   
3                                                    None           None   
4                                                    None           None   
...                                                   ...            ...   
116860  SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR ...           None   
116861                                               None           None   
116862                                               None           None   
116863                                               None           None   
116864  End of this Etext of The Complete Works of Wil...           None   

       string_field_2 string_field_3  
0                N

In [104]:
# drop columns and clean up rows with none
import pandas as pd
df1 = df.drop(['string_field_1','string_field_2','string_field_3'],axis=1)
df2 = df1.rename(columns={'string_field_0':'text'})
df3 = df2.dropna()
df3.head

<bound method NDFrame.head of                                                      text
0                                             THE SONNETS
2                                  by William Shakespeare
6                                                       1
7               From fairest creatures we desire increase
8              That thereby beauty's rose might never die
...                                                   ...
116857  DISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR...
116858                                  PERSONAL USE ONLY
116859  COMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUT...
116860  SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR ...
116864  End of this Etext of The Complete Works of Wil...

[107829 rows x 1 columns]>

In [105]:
# covert caps to lower case and clean the punctuation 
ds=df3['text'].str.lower().str.replace(r'[^\w\s]+', '')
df4 = pd.DataFrame({'text':ds})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [106]:
# now split the strings into lists 
df5 = df4.assign(text=df4.text.str.split(" "))
df5.head

<bound method NDFrame.head of                                                      text
0                                          [the, sonnets]
2                              [by, william, shakespeare]
6                                                     [1]
7        [from, fairest, creatures, we, desire, increase]
8       [that, thereby, beautys, rose, might, never, die]
...                                                   ...
116857  [distributed, so, long, as, such, copies, 1, a...
116858                              [personal, use, only]
116859  [commercially, , prohibited, commercial, distr...
116860  [service, that, charges, for, download, time, ...
116864  [end, of, this, etext, of, the, complete, work...

[107829 rows x 1 columns]>

In [107]:
# now explode the lists into separate rows
df6 = df5.explode('text').reset_index(drop=True)
df6.head

<bound method NDFrame.head of                text
0               the
1           sonnets
2                by
3           william
4       shakespeare
...             ...
656203     complete
656204        works
656205           of
656206      william
656207  shakespeare

[656208 rows x 1 columns]>

In [108]:
# now count the words
final_df = df6.value_counts().reset_index()
final_df.columns = ['word','count']
final_df.head

<bound method NDFrame.head of            word  count
0                25660
1           the  20113
2           and  16251
3            to  13932
4            of  13729
...         ...    ...
24383    madame      1
24384  madbrain      1
24385   madbred      1
24386   madeira      1
24387  zwaggerd      1

[24388 rows x 2 columns]>